# Anneal Schedule
This notebook explores features that give you control over the D-Wave system's global [anneal schedule](#Understanding-the-Anneal-Schedule). These features can improve solutions to a problem and provide insight into the behaviour and dynamics of problems undergoing quantum annealing.

1. [Understanding the Anneal Schedule](#Understanding-the-Anneal-Schedule) explains the feature.
2. [Using Anneal Schedule Features](#Using-Anneal-Schedule-Features) shows how to use the feature with an interactive example problem.
3. [Mapping Various Anneal Schedules](#Mapping-Various-Anneal-Schedules-on-the-QPU) provides code that sweeps through various anneal schedules to explore the effect on results.

Uses of the feature are described, for example, in [The power of pausing](https://arxiv.org/abs/1810.05881) 
<div class="alert alert-warning" role="alert" style="margin: 10px">
Note that the final section of this Jupyter Notebook sends a large number of problems to the QPU, takes several minutes to complete, and uses tens of seconds of QPU time. If your account has a limited quota, consider running only the first sections (and not using the Run All menu option).   
</div>

**New to Jupyter Notebooks?** JNs are divided into text or code cells. Pressing the **Run** button in the menu bar moves to the next cell. Code cells are marked by an "In: \[\]" to the left; when run, an asterisk displays until code completion: "In: \[\*\]".

## Feature Availability

Availability of the anneal schedule features depends on the solver you connect to. To see whether your solver supports anneal schedule features, check solver property `max_anneal_schedule_points`  and ensure it is at least 4. 

First, we set up a connection to the quantum processing unit (QPU). Enter your token value if required.

In [ ]:
from dwave.system.samplers import DWaveSampler
sampler = DWaveSampler(solver={'qpu': True})
print("Connected to sampler", sampler.solver.id)

In [ ]:
print("Maximum anneal schedule points: {}".format(sampler.properties["max_anneal_schedule_points"]))

Your code can ensure a solver supports the feature by making it a selection criteria: `sampler = DWaveSampler(solver={'qpu': True, 'max_anneal_schedule_points__gte': 4})` selects only a QPU that supports at least 4 anneal-schedule points.

## Understanding the Anneal Schedule

The [Getting Started with the D-Wave System](https://docs.dwavesys.com/docs/latest/doc_getting_started.html) document describes quantum annealing and the [Solver Properties and Parameters](https://docs.dwavesys.com/docs/latest/doc_solver_ref.html) book describes the parameters used here.

In brief, *anneal schedule* refers to the global annealing trajectory. It specifies the normalized anneal fraction, $s$, an abstract parameter ranging from 0 to 1. $s(t)$ is a continuous function starting at $s=0$ for time $t=0$ and ending with $s=1$ at $t=t_f$, the total time of the anneal.

There are two ways to specify the anneal schedule, using two *mutually exclusive* parameters:

1. `annealing_time`: Set to a number in microseconds to specify linear growth from $s=0$ to $s=1$ over that time.
2. `annealing_schedule`: Specify a list of ($t$, $s$) pairs specifying points, which are then linearly interpolated. This feature supports two modes &mdash; mid-anneal *pause* and mid-anneal *quench* &mdash; which this tutorial explores.
   
<div class="alert alert-warning" role="alert" style="margin: 10px">
It is an error to specify both `annealing_time` and `annealing_schedule`.
</div>

The slope of a line segment is specified as $m_q = \frac{\Delta s}{\Delta t}$, with units of $\mu s^{-1}$. The maximum slope is equal to the inverse of the minimum anneal time. For example, a quench slope of $m_q = 0.5$ is equivalent to a 2 $\mu s$ standard anneal. The cell below calculates the maximum slope for your sampler.

In [ ]:
annealing_range = sampler.properties["annealing_time_range"]
max_slope = 1.0/annealing_range[0]
print("Annealing time range: {}".format(sampler.properties["annealing_time_range"]))
print("Maximum slope:", max_slope)

A number of rules must be followed when building an anneal schedule:

* The first point must be (0,0).
* Normalized anneal fraction $s$ must increase monotonically.
* In the final point, $s$ must equal 1 and time $t$ must not exceed the maximum value in the `annealing_time_range` property.
* The number of points must be $\ge 2$. The upper bound is system-dependent &mdash; check the `max_anneal_schedule_points` property.
* The slopes of each line segment must not violate the maximum slope $m_{max}$.

Additional rules that govern maximum slope vary by system; check the QPU properties document for your system. 

<div class="alert alert-warning" role="alert" style="margin: 10px">
Violating these rules returns an error.
</div>

The sections below demonstrate three schedule variations: 

* [Pause](#Pause) 
* [Quench](#Quench) 
* [Pause and Quench](#Pause-and-Quench)

### Pause

This section creates an anneal schedule with a pause of 200 $\mu s$ halfway through a 100 $\mu s$ anneal. The following code specifies four points: (1) start of anneal, (2) beginning of pause, (3) end of pause, and (4) end of anneal.

(1) Anneal begins at point $[0, 0]$. 

(2 & 3) Anneal continues until half-way through a $100 \mu s$ anneal; that is, time $50 \mu s$ with $s = 0.5$ (half-way to an endpoint that must have $s=1.0$). So the next two schedule points are $[50, 0.5]$ and $[250,0.5]$ for a pause of $200 \mu s$.

(4) Aneal resumes for the last $50 \mu s$ and reaches $s=1.0$, making the endpoint $[300, 1]$.

The next cell plots the anneal schedule with a pause.

In [ ]:
# Use NumPy and Bokeh for plotting
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

# Print the full anneal schedule
schedule = [[0.0,0.0], [50.0, 0.5], [250.0, 0.5], [300, 1.0]]
print("Schedule: %s" % schedule)

# Plot the anneal schedule
p = figure(title="Example Anneal Schedule with Pause", x_axis_label='Time [us]', y_axis_label='Annealing Parameter s')
p.line(*np.array(schedule).T)
show(p)

### Quench

This section creates an anneal schedule with a quench. At 60% of the way through a 20 $\mu s$ anneal, it sets a slope equivalent to a 2 $\mu s$ anneal. This schedule has three points:

(1) The schedule starts at $[0, 0]$.

(2) The next schedule point is 60% of the way through a 20 $\mu s$ anneal; that is, at 12 $\mu s$. To determine $s$ at time $t = 12 \mu s$, consider the slope of a standard $20 \mu s$ anneal. Recall that the slope is defined as $m_q = \frac{\Delta s}{\Delta t}$, so the slope for the first 12 $\mu s$ is $\frac{1}{20}$ and the next schedule point is $[12, 0.6]$.

(3) The quench slope is equivalent to a 2 $\mu s$ anneal, which has slope $\frac{1}{2}$. This last segment must end with $s(t)=1$, which occurs at time $t=12.8 \mu s$. This makes the last schedule point $[12.8, 1]$.

The next cell plots the anneal schedule with a quench.

In [ ]:
schedule=[[0.0,0.0],[12.0,0.6],[12.8,1.0]]
print("Schedule: %s" % schedule)
p = figure(title="Example Anneal Schedule with Quench", x_axis_label='Time [us]', y_axis_label='Annealing Parameter s')
p.line(*np.array(schedule).T)
show(p)

### Pause and Quench

This section creates an anneal schedule with a quench following a 50 $\mu s$ pause 40% of the way through a 100 $\mu s$ anneal. As in section [Pause](#Pause), the schedule needs four points. Again, the resulting slope must not exceed the maximum slope for your system.

The next cell plots the anneal schedule with a pause and a quench.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
schedule=[[0.0, 0.0], [40.0, 0.4], [90.0, 0.4], [91.2, 1.0]]
print("Schedule: %s" % schedule)
p = figure(title="Example Anneal Schedule with Pause and Quench", x_axis_label='Time [us]', y_axis_label='Annealing Parameter s')
p.line(*np.array(schedule).T)
show(p)

## Using Anneal Schedule Features
This section demonstrates how the pause and quench features can improve solutions for an example problem. We use the 16-qubit system shown below, which was studied in https://www.nature.com/articles/ncomms2920, and has an energy gap between the classical ground state and excited states of 4.

This system is set up as shown in the image below, where dots are qubits with colors representing bias values and lines representing couplings (couplings weights are $-1$).
 
<img src='images/16q_system.png'>

The structure of this problem maps neatly to the Chimera topology of a D-Wave QPU: it can be minor-embedded directly into two side-by-side Chimera unit cells, with each problem qubit represented by one qubit on the QPU. If you were to use a standard heuristic embedder such as [minorminer](https://docs.ocean.dwavesys.com/projects/minorminer/en/latest/), however, some of the problem qubits might be represented by chains of qubits on the QPU.  
Instead, the next cell scans adjacent unit cells on the QPU and finds the first pair in which all the required qubits and couplers are available and provides a one-to-one minor-embedding. 

In [ ]:
# To see helper functions, select Jupyter File Explorer View from the Online Learning page
from helpers.embedding import direct_embedding
nodes, edges = direct_embedding(sampler.nodelist, sampler.edgelist)

The next cell formulates the problem for the D-Wave system (configures values of the $h_i$ and $J_{i,j}$ linear and quadratic coefficients used to program the system).

In [ ]:
h = dict(zip(nodes, [1., -1., -1., 1., 1., -1., 0., 1., 1., -1., -1., 1., 1., 0., -1., 1.]))
J = {edge:-1 for edge in edges}
print("h: ", h, "\n")
print("J: ", J)

For reference, the next cell runs the problem with a default anneal schedule.

In [ ]:
runs=1000
results = sampler.sample_ising(h, J, 
    num_reads=runs, 
    answer_mode='raw',
    annealing_time=100)

print("QPU time used:", results.info['timing']['qpu_access_time'], "microseconds.")

The following visualization is a histogram of the returned solutions. Typically, the ground state (energy = -20.0) is found in just a fraction of the samples with the default anneal schedule.

In [ ]:
plt.hist(results.record.energy,rwidth=1,align='left',bins=[-21,-20,-19,-18,-17,-16,-15])
plt.show()

The next cell computes the percentage of ground-state solutions in the returned samples. You can infer the number of required samples you would need to ensure with high probability a good solution to the problem.

In [ ]:
_, counts = np.unique(results.record.energy.reshape(1000,1), axis=0, return_counts=True)
print("Ground state probability: ", counts[0]/runs)

### Using Pause

This section shows some anneal schedules that include a pause. For three different anneal times and three different pause times, ground-state probability is measured as a function of $s_{pause}$, the fraction of the anneal time where the pause occurs. Success rate should increase when the pause occurs near to the freeze-out time.

The schedules used here are all combinations of the following:

  1. Anneal times of 10, 100, or 500 $\mu s$.

  2. Pause duration of 10, 100, or 500 $\mu s$.

  3. Pause starting at various points of $s$ between 0.2 to 0.6.

This section uses saved results from a file. Section [Running the Anneals on the QPU](#Running-the-Anneals-on-the-QPU) provides the code for you to run these anneals yourself. Note that running these anneals sends a large number of problems to the QPU, so that section takes several minutes to complete and uses a large number of seconds of QPU time.

In [ ]:
import pandas as pd
import json
with open("files/saved_pause_results.json", "r") as read_file:
    saved_pause_success_prob = pd.read_json(json.load(read_file))

The next cell plots the results:

In [ ]:
from itertools import cycle

# set up some plot decoration values
all_colors = ["dodgerblue", "orange", "darkorchid"]
all_lines = ["4 1 4", "4 4", "4 0"]
all_markers = ["circle", "square", "triangle", "diamond"]

p = figure(title="Success fraction using pause for a range of anneal schedule parameters",
           x_axis_label='s at pause start',
           y_axis_label='ground state fraction',
           y_axis_type='log',
           y_range=[1e-3, 0.65],
           plot_height=960,
           plot_width=960)

colors = cycle(all_colors)
lines = cycle(all_lines)
for anneal,a_group in saved_pause_success_prob.groupby('anneal_time'):
    col = next(colors)
    line = next(lines)
    
    markers = cycle(all_markers)
    for pause,p_group in a_group.groupby('pause_duration'):
        marker = next(markers)
        legend = "anneal={}, pause={}".format(anneal,pause)
        
        x = p_group["s_feature"]
        y = p_group["success_frac"]
        p.line(x, y, legend=legend, color=col, line_dash=line, line_width=2)
        p.scatter(x, y, legend=legend, color=col, marker=marker, size=8)
    
p.legend.location = "top_left"
show(p)

<div class="alert alert-block alert-success">
<b>Exercise:</b> Based on the results above, can you find a better ground state fraction? Define your own anneal schedule in the code block below and see how your results compare.
</div>

Your new point appears as a red circle on the plot. Each time you run this next cell a new red point is added to your previous points.

In [ ]:
#Update the standard anneal schedule parameters below: 

anneal_time = 10.0
pause_duration = 500.0      # Must be greater than 0
pause_start = 0.4        # Must be between 0 and 1

#----------------------------------------------------------------
#Leave the code below to run the problem and display the results.
#----------------------------------------------------------------
schedule=[[0.0,0.0],[pause_start*anneal_time,pause_start],[pause_start*anneal_time+pause_duration, pause_start],[anneal_time+pause_duration, 1.0]]
runs=900
results = sampler.sample_ising(h, J,
                anneal_schedule=schedule,
                num_reads=runs, 
                answer_mode='raw',
                num_spin_reversal_transforms=1)
success = np.count_nonzero(results.record.energy == -20.0)/runs
print("Success probability: ",success)

p.scatter([pause_start],[success], color="red")
show(p)

### Quench
This section looks at the effects of using a mid-anneal quench. As above, it sweeps through a range of anneal schedule parameters. Success rates should be low when the quench occurs before freeze out and be similar to a standard anneal when the quench occurs after freeze out.

This section uses saved results from a file. Section [Running the Anneals on the QPU](#Running-the-Anneals-on-the-QPU) provides the code for you to run these anneals yourself. Note that running these anneals sends a large number of problems to the QPU, so that section takes several minutes to complete and uses a large number of seconds of QPU time.

In [ ]:
with open("files/saved_quench_results.json", "r") as read_file:
    saved_quench_success_prob = pd.read_json(json.load(read_file))

The next cell plots the results.

In [ ]:
p = figure(title="Success fraction using quench for a range of anneal schedule parameters",
           x_axis_label='s at quench start',
           y_axis_label='ground state fraction',
           y_axis_type='log',
           y_range=[1e-4, 0.5],
           plot_height=960,
           plot_width=960)

colors = cycle(all_colors)
lines = cycle(all_lines)
for anneal,a_group in saved_quench_success_prob.groupby('anneal_time'):
    col = next(colors)
    line = next(lines)
   
    markers = cycle(all_markers)
    for quench,p_group in a_group.groupby('quench_slope'):
        marker = next(markers)
        legend = "anneal={}, quench slope={}".format(anneal,quench)
        
        x = p_group["s_feature"]
        y = p_group["success_frac"]
        p.line(x, y, legend=legend, color=col, line_dash=line, line_width=2)
        p.scatter(x, y, legend=legend, color=col, marker=marker, size=8)

p.legend.location = "top_left"
show(p)

<div class="alert alert-block alert-success">
<b>Exercise:</b> Based on the results above, can you find a better ground state fraction? Define your own anneal schedule in the code block below and see how your results compare.
</div>

Your new point appears as a red circle on the plot. Each time you run this next cell a new red point is added to your previous points.

In [ ]:
#Update the standard anneal schedule parameters below 

anneal_time = 50.0
quench_slope = 1.0      # Must be greater than 0
quench_start = 0.2      # Must be between 0 and 1

#----------------------------------------------------------------
#Leave the code below to run the problem and display the results.
#----------------------------------------------------------------
schedule=[[0.0,0.0],[quench_start*anneal_time,quench_start],[(1-quench_start+quench_slope*quench_start*anneal_time)/quench_slope, 1.0]]
runs=950
results = sampler.sample_ising(h, J,
                anneal_schedule=schedule,
                num_reads=runs, 
                answer_mode='raw',
                num_spin_reversal_transforms=1)
success = np.count_nonzero(results.record.energy == -20.0)/runs
print("Success probability: ",success)

p.scatter([quench_start],[success], color="red")
show(p)

# Mapping Various Anneal Schedules  on the QPU
Running the code in this section uses tens of seconds of QPU time. You can copy these cells to the respective cells in the previous section that use saved results. 

## Running the Paused Anneals on the QPU

<div class="alert alert-warning" role="alert" style="margin: 10px">
Note that the next cell sends a large number of problems to the QPU, so running takes several minutes to complete and uses a large number of seconds of QPU time.
</div>

In [ ]:
anneal_time=[10.0, 100.0, 500.0]
pause_duration=[10.0, 100.0, 500.0]

# Create list of start times
num_points=5
s_low=0.2
s_high=0.6
pause_start=np.linspace(s_low, s_high, num=num_points)

success_prob = pd.DataFrame(index=range(len(anneal_time)*len(pause_duration)*len(pause_start)),
                                columns=["anneal_time", "pause_duration", "s_feature", "success_frac"],
                                data=None)
counter=0

print("Starting QPU calls...")
QPU_time=0.0
for anneal in anneal_time:
    for pause in pause_duration:
        for start in pause_start:
            schedule=[[0.0,0.0],[start*anneal,start],[start*anneal+pause, start],[anneal+pause, 1.0]]
            runs=950
            results = sampler.sample_ising(h, J,
                anneal_schedule=schedule,
                num_reads=runs, 
                answer_mode='raw',
                num_spin_reversal_transforms=1)
            success_prob.iloc[counter]={"anneal_time":anneal, 
                                        "pause_duration":pause, 
                                        "s_feature":start,
                                        "success_frac":np.count_nonzero(results.record.energy == -20.0)/runs}
            counter+=1
            QPU_time+=results.info['timing']['qpu_access_time']
        print("QPU calls remaining: ", len(anneal_time)*len(pause_duration)*len(pause_start)-counter)
            
print("QPU calls complete using", QPU_time/1000000.0, "seconds of QPU time.")

## Running the Quenched Anneals on the QPU

<div class="alert alert-warning" role="alert" style="margin: 10px">
Note that the next cell sends a large number of problems to the QPU, so running takes several minutes to complete and uses a large number of seconds of QPU time.
</div>

In [ ]:
anneal_time=[10.0, 100.0, 500.0]
quench_slopes=[1.0, 0.5, 0.25]

# start times
num_points=5
s_low=0.2
s_high=0.9
quench_start=np.linspace(s_low, s_high, num=num_points)

success_prob = pd.DataFrame(index=range(len(anneal_time)*len(quench_slopes)*len(quench_start)),
                                columns=["anneal_time", "quench_slope", "s_feature", "success_frac"],
                                data=None)
counter=0

print("Starting QPU calls...")
QPU_time=0.0
for anneal in anneal_time:
    for quench in quench_slopes:
        for start in quench_start:
            schedule=[[0.0,0.0],[start*anneal,start],[(1-start+quench*start*anneal)/quench, 1.0]]
            runs=1000
            results = sampler.sample_ising(h, J,
                anneal_schedule=schedule,
                num_reads=runs, 
                answer_mode='raw',
                num_spin_reversal_transforms=1)
            success_prob.iloc[counter]={"anneal_time":anneal, 
                                        "quench_slope":quench, 
                                        "s_feature":start,
                                        "success_frac":np.count_nonzero(results.record.energy == -20.0)/runs}
            counter+=1
            QPU_time+=results.info['timing']['qpu_access_time']
        print("QPU calls remaining: ", len(anneal_time)*len(quench_slopes)*len(quench_start)-counter)
            
print("QPU calls complete using", QPU_time/1000000.0, "seconds of QPU time.")